In [ ]:

#!pip install sqlalchemy==2.0
#!pip install ipython-sql

In [15]:
#load the sql extension 
%load_ext sql

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [16]:
#establish a connection with the database
import csv, sqlite3

con = sqlite3.connect("my_data1.db")
cur = con.cursor()

In [17]:
%sql sqlite:///my_data1.db

In [18]:
import pandas as pd
df = pd.read_csv("Spacex.csv")
df.to_sql("SPACEXTBL", con, if_exists='replace', index=False,method="multi")

101

In [19]:
%sql create table SPACEXTABLE as select * from SPACEXTBL where Date is not null

 * sqlite:///my_data1.db
Done.


[]

In [20]:
#Display the names of the unique launch sites in the space mission
%sql select distinct("Launch_Site") from SPACEXTABLE

 * sqlite:///my_data1.db
Done.


Launch_Site
CCAFS LC-40
VAFB SLC-4E
KSC LC-39A
CCAFS SLC-40


In [21]:
#Display 5 records where launch sites begin with the string 'CCA'
%sql select * from SPACEXTABLE where "Launch_Site" like "CCA%" LIMIT 5

 * sqlite:///my_data1.db
Done.


Date,Time (UTC),Booster_Version,Launch_Site,Payload,PAYLOAD_MASS__KG_,Orbit,Customer,Mission_Outcome,Landing_Outcome
2010-06-04,18:45:00,F9 v1.0 B0003,CCAFS LC-40,Dragon Spacecraft Qualification Unit,0,LEO,SpaceX,Success,Failure (parachute)
2010-12-08,15:43:00,F9 v1.0 B0004,CCAFS LC-40,"Dragon demo flight C1, two CubeSats, barrel of Brouere cheese",0,LEO (ISS),NASA (COTS) NRO,Success,Failure (parachute)
2012-05-22,7:44:00,F9 v1.0 B0005,CCAFS LC-40,Dragon demo flight C2,525,LEO (ISS),NASA (COTS),Success,No attempt
2012-10-08,0:35:00,F9 v1.0 B0006,CCAFS LC-40,SpaceX CRS-1,500,LEO (ISS),NASA (CRS),Success,No attempt
2013-03-01,15:10:00,F9 v1.0 B0007,CCAFS LC-40,SpaceX CRS-2,677,LEO (ISS),NASA (CRS),Success,No attempt


In [23]:
#Display the total payload mass carried by boosters launched by NASA (CRS)
%sql select sum("PAYLOAD_MASS__KG_") as total_payload_mass from SPACEXTABLE where Customer like 'NASA (CRS)%'

 * sqlite:///my_data1.db
Done.


total_payload_mass
48213


In [28]:
#Display average payload mass carried by booster version F9 v1.1
%sql select avg("PAYLOAD_MASS__KG_") from SPACEXTABLE where Booster_Version == "F9 v1.1"

 * sqlite:///my_data1.db
Done.


"avg(""PAYLOAD_MASS__KG_"")"
2928.4


In [26]:
%sql select distinct(Landing_Outcome) from SPACEXTABLE

 * sqlite:///my_data1.db
Done.


Landing_Outcome
Failure (parachute)
No attempt
Uncontrolled (ocean)
Controlled (ocean)
Failure (drone ship)
Precluded (drone ship)
Success (ground pad)
Success (drone ship)
Success
Failure


In [27]:
#List the date when the first succesful landing outcome in ground pad was acheived
%sql select min(Date) from SPACEXTABLE where Landing_Outcome == 'Success (ground pad)'

 * sqlite:///my_data1.db
Done.


min(Date)
2015-12-22


In [32]:
#List the names of the boosters which have success in drone ship and have payload mass greater than 4000 but less than 6000
%sql select Booster_Version from SPACEXTABLE where Landing_Outcome == 'Success (drone ship)' and "PAYLOAD_MASS__KG_" > 4000 and "PAYLOAD_MASS__KG_" < 6000 

 * sqlite:///my_data1.db
Done.


Booster_Version
F9 FT B1022
F9 FT B1026
F9 FT B1021.2
F9 FT B1031.2


In [ ]:
%sql select distinct(Mission_Outcome) from SPACEXTABLE

In [38]:
#List the total number of successful and failure mission outcomes
%sql select Mission_Outcome, count("Mission_Outcome") as count from SPACEXTABLE group by Mission_Outcome

 * sqlite:///my_data1.db
Done.


Mission_Outcome,count
Failure (in flight),1
Success,98
Success,1
Success (payload status unclear),1


In [39]:
#List the names of the booster_versions which have carried the maximum payload mass. Use a subquery
%sql select Booster_Version from SPACEXTABLE WHERE PAYLOAD_MASS__KG_ == (select max(PAYLOAD_MASS__KG_) from SPACEXTABLE)

 * sqlite:///my_data1.db
Done.


Booster_Version
F9 B5 B1048.4
F9 B5 B1049.4
F9 B5 B1051.3
F9 B5 B1056.4
F9 B5 B1048.5
F9 B5 B1051.4
F9 B5 B1049.5
F9 B5 B1060.2
F9 B5 B1058.3
F9 B5 B1051.6


In [41]:
#List the records which will display the month names, failure landing_outcomes in drone ship ,booster versions, launch_site for the months in year 2015
%sql select substr(Date, 6,2) as Month, "Landing_Outcome","Booster_Version","Launch_Site" from SPACEXTABLE where substr(Date,0,5)='2015' and "Landing_Outcome" == 'Failure (drone ship)'

 * sqlite:///my_data1.db
Done.


Month,Landing_Outcome,Booster_Version,Launch_Site
01,Failure (drone ship),F9 v1.1 B1012,CCAFS LC-40
04,Failure (drone ship),F9 v1.1 B1015,CCAFS LC-40


In [ ]:
#Rank the count of landing outcomes (such as Failure (drone ship) or Success (ground pad)) between the date 2010-06-04 and 2017-03-20, in descending order.
%sql select Landing_Outcome, count(Landing_Outcome) as Rank from SPACEXTABLE where Date between '2010-06-04' and '2017-03-20' group by "Landing_Outcome" order by count("Landing_Outcome") desc

In [49]:
%sql select "Landing_Outcome", count("Landing_Outcome") as count, RANK() OVER (ORDER BY count("Landing_Outcome") desc) as xRank from SPACEXTABLE where Date between '2010-06-04' and '2017-03-20' group by "Landing_Outcome" order by count("Landing_Outcome") desc

 * sqlite:///my_data1.db
Done.


Landing_Outcome,count,xRank
No attempt,10,1
Success (drone ship),5,2
Failure (drone ship),5,2
Success (ground pad),3,4
Controlled (ocean),3,4
Uncontrolled (ocean),2,6
Failure (parachute),2,6
Precluded (drone ship),1,8
